In [9]:
from project.models import Project
from public_data.models import Ocsge, Departement
from django.db.models import Max
from ipyleaflet import Map, GeoJSON
from rest_framework_gis.serializers import GeoFeatureModelSerializer
from django.contrib.gis.db.models import Union

class SinglePolySerializer(GeoFeatureModelSerializer):
    class Meta:
        model = RefPlan
        geo_field = "mpoly"
        fields=()


In [12]:
all_ocsge_mpoly = Ocsge.objects.aggregate(mpoly=Union("mpoly"))["mpoly"]

In [2]:
Project.objects.all()

<QuerySet [<Project: Diagnostic de CA Grand Auch Coeur de Gascogne>]>

In [3]:
p = Project.objects.all().first()

In [4]:
Ocsge.objects.filter(mpoly__intersects=p.combined_emprise)

<QuerySet [<Ocsge: Ocsge object (364623)>, <Ocsge: Ocsge object (365274)>, <Ocsge: Ocsge object (365282)>, <Ocsge: Ocsge object (366848)>, <Ocsge: Ocsge object (365290)>, <Ocsge: Ocsge object (364986)>, <Ocsge: Ocsge object (366978)>, <Ocsge: Ocsge object (364861)>, <Ocsge: Ocsge object (364914)>, <Ocsge: Ocsge object (364744)>, <Ocsge: Ocsge object (364731)>, <Ocsge: Ocsge object (364909)>, <Ocsge: Ocsge object (364221)>, <Ocsge: Ocsge object (367442)>, <Ocsge: Ocsge object (364483)>, <Ocsge: Ocsge object (362660)>, <Ocsge: Ocsge object (364244)>, <Ocsge: Ocsge object (367444)>, <Ocsge: Ocsge object (355433)>, <Ocsge: Ocsge object (364230)>, '...(remaining elements truncated)...']>

In [5]:
(
    p.cities.all()
    .first()
    .communediff_set.all()
    .aggregate(old=Max("year_old"), new=Max("year_new"))
)

{'old': 2016, 'new': 2019}

In [6]:
Departement.objects.all().filter(is_artif_ready=True)

<QuerySet []>

In [7]:
test_filters = ["coveredby", "within", "contained", "crosses", "disjoint", "intersects", "overlaps"]
depts = []
for dept in Departement.objects.all():
    qs = Ocsge.objects.filter(mpoly__contained=dept.mpoly)
    if qs.exists():
        depts.append(dept)
depts

[<Departement: Haute-Garonne>,
 <Departement: Gers>,
 <Departement: Gironde>,
 <Departement: Landes>,
 <Departement: Lot-et-Garonne>,
 <Departement: Pyrénées-Atlantiques>,
 <Departement: Hautes-Pyrénées>,
 <Departement: Tarn-et-Garonne>]

In [13]:

dept = Departement(
    source_id="000",
    region_id=1,
    name="test",
    mpoly=all_ocsge_mpoly,
)



center = (dept.mpoly.centroid.y, dept.mpoly.centroid.x)
m = Map(center=center, zoom=5)

m.add_layer(GeoJSON(
    data=SinglePolySerializer(dept).data,
    style={'color': 'red', 'fillOpacity': 0.3, 'weight': 1},
))

serializer = SinglePolySerializer(depts, many=True)
m.add_layer(GeoJSON(
    data=serializer.data,
    style={'color': 'yellow', 'fillOpacity': 0.3, 'weight': 1},
))

m

Map(center=[43.89439817145893, 0.14285194467326362], controls=(ZoomControl(options=['position', 'zoom_in_text'…